In [18]:
import torch
import numpy as np
from torchvision import transforms as T
from lightly.transforms import SimCLRTransform, DINOTransform, MAETransform, MoCoV2Transform, utils
from datasets import create_dataset, create_bootstrap_dataloader
from models import *
from tqdm import tqdm
import pytorch_lightning as pl
import os
import copy
import gc
import matplotlib.pyplot as plt

In [20]:
%load_ext autoreload
%autoreload 2

# Losowość

In [3]:
SEED = 42

def seed_everything(seed: int=42):
    pl.seed_everything(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    ####### Normaly you would also need to seed those generators but `pytorch_lightning` does it in one func
    # random.seed(seed)
    # np.random.seed(seed)
    # torch.manual_seed(seed)
    ######
    torch.cuda.manual_seed(seed) # Don't know if pytorch lightning does this
    torch.cuda.manual_seed_all(seed) # Don't know if pytorch lightning does this
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


seed_everything(SEED)

Seed set to 42


# Transformacje

In [4]:
test_transform = T.v2.Compose(
    [
        T.Resize((224, 224)),
        T.v2.ToImage(),
        T.v2.ToDtype(torch.float32, scale=True),
        T.Normalize(
            mean=utils.IMAGENET_NORMALIZE["mean"],
            std=utils.IMAGENET_NORMALIZE["std"],
        )
    ]
)
scratch_transform = T.v2.Compose(
    [
        T.RandomResizedCrop((224, 224)),
        T.RandomHorizontalFlip(),
        T.v2.ToImage(),
        T.v2.ToDtype(torch.float32, scale=True),
        T.Normalize(
            mean=utils.IMAGENET_NORMALIZE["mean"],
            std=utils.IMAGENET_NORMALIZE["std"],
        )
    ]
)

byol_transform = SimCLRTransform(input_size=(224, 224), vf_prob=0.5, rr_prob=0.5) #BYOL używa SimCLR-owych transformacji

/home/hussein/pytoniec/lib/python3.13/site-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


# Datasety

In [5]:
train_full_ds, train_ssl_ds, train_ds, test_ds = create_dataset("ImageNet1K", 0.9, byol_transform, scratch_transform, test_transform, "data", False)

Length of entire train dataset:  1281167
Length of SSL train dataset:  1153050
Length of classification train dataset:  128117
Length of test dataset:  50000


# Hiperparametry

In [23]:
### PARAMETERS ###
BATCH_SIZE = 128
NUM_EPOCHS = 1
LEARNING_RATE = 0.001
NUM_WORKERS = 3
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")
#GARBAGE COLLECTOR FAJNA SPRAWA - BEZ NIEGO VRAMu BRAKUJE
if device == "gpu":
    torch.cuda.empty_cache()
    gc.collect()

Using device: cuda


# Uczenie na resnet18 (bootstrapowe datasety pragnę zauważyć, stratified znaczy, że stara się balansować próbkowanie po klasach)

In [12]:
# jednak nie będzie stratified bo ekstrakcja tego ma zając 2,5h xd ( te targety są wyliczane w funkcji create_dataset, można ją zmodyfikować tak aby zwracała
labels = np.array([y for _, y in tqdm(train_ssl_ds)])

  1%|          | 12497/1153050 [01:37<2:29:03, 127.53it/s]


KeyboardInterrupt: 

# Uczenie na pustym resnet18

# Uczenie na jakiejś prostej MLP

In [25]:
dl_train = create_bootstrap_dataloader(
    dataset=train_ssl_ds,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    random_state=SEED,

    pin_memory=True,
    drop_last=True)
dl_val = torch.utils.data.DataLoader(
    test_ds,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS
)

#backbone_type - simple_mlp - nie używamy resneta
dino_model = BYOLModel(lr=LEARNING_RATE, max_epochs=NUM_EPOCHS, backbone_type='simple_mlp')

trainer = pl.Trainer(accelerator='cuda', devices=1, max_epochs=NUM_EPOCHS, precision="16-mixed", log_every_n_steps=1)

trainer.fit(dino_model, dl_train, dl_val)

Using 16bit Automatic Mixed Precision (AMP)
Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | Type                 | Params | Mode 
-------------------------------------------------------------------
0 | online_backbone   | SimpleMLP            | 19.3 M | train
1 | online_projection | SimCLRProjectionHead | 33.3 K | train
2 | online_predictor  | Sequential           | 16.7 K | train
3 | target_backbone   | SimpleMLP            | 19.3 M | train
4 | target_projection | SimCLRProjectionHead | 33.3 K | train
-------------------------------------------------------------------
19.4 M    Trainable params
19.3 M    Non-trainable params
38.7 M    Total params
154.743   Total estimated model params size (MB)
4

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.


# zostawiam output poniższej komórki, wydaje mi się zabawny

In [22]:
dl_train = create_bootstrap_dataloader(
    dataset=train_ssl_ds,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    random_state=SEED,

    pin_memory=True,
    drop_last=True)
dl_val = torch.utils.data.DataLoader(
    test_ds,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS
)

#backbone_type - simple_mlp - nie używamy resneta
dino_model = BYOLModel(lr=LEARNING_RATE, max_epochs=NUM_EPOCHS, backbone_type='simple_mlp')

trainer = pl.Trainer(accelerator='cuda', devices=1, max_epochs=NUM_EPOCHS, precision="16-mixed", log_every_n_steps=1)

trainer.fit(dino_model, dl_train, dl_val)

Using 16bit Automatic Mixed Precision (AMP)
Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | Type                 | Params | Mode 
-------------------------------------------------------------------
0 | online_backbone   | SimpleMLP            | 310 M  | train
1 | online_projection | SimCLRProjectionHead | 33.3 K | train
2 | online_predictor  | Sequential           | 16.7 K | train
3 | target_backbone   | SimpleMLP            | 310 M  | train
4 | target_projection | SimCLRProjectionHead | 33.3 K | train
-------------------------------------------------------------------
311 M     Trainable params
311 M     Non-trainable params
622 M     Total params
2,488.167 Total estimated model params size (MB)
5

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]


Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined